# In this program, we will be performing minimum mass estimates on the remaining objects from the DR17 APOGEE catalog.

## Notes: our radii valuses are from Yu et al. 2023 (https://arxiv.org/abs/2206.00046), and our Joker parameters come from the APOGEE Value Added Catalog by Price-Whelan et al. 2017 (https://thejoker.readthedocs.io/en/latest/). We also calculated stellar companion mass estimates using the open-source package kiauhoku (https://arxiv.org/abs/1911.04518 // https://github.com/zclaytor/kiauhoku)
## Using two separate procedures (ref: Schochet et al. in prep) we use the binary mass function to estimate the mass of hidden companions to the systems below

In [29]:
#First, importing the necessary packages

import astropy
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.table import Table
import re

In [30]:
#Import CSV Files of objects which include radii (Yu et al. 2023) and Kiauhoku estimates using Dart/MIST/GARS/and YREC isochrones

df_fircomb = pd.read_csv(r'/Users/mschochet/Desktop/Research/Testing_Docs/apogee_objects_with_kiauhoku_masses.csv')
df_fircomb

,Unnamed: 0,Field,Telescope,SDSS ID,Gaia ID,nvisits,SNR,RA,Dec,Star Flags (bitwise),...,flag_yrec,gars_mass,age_gars,flag_gars,mist_mass,age_mist,flag_mist,dart_mass,age_dart,flag_dart
0,0,120+12,apo25m,2M00000662+7528598,539684085518533504,9,773.235000,0.027622,75.483292,131072,...,2,0.999707,5.054063,2,0.999854,4.235257,2,1.000000,5.116467,2
1,1,100-60,apo25m,2M00005343+0040594,2738248909142217600,4,95.417015,0.222657,0.683168,8390656,...,0,1.038642,1.991835,0,1.002548,2.727639,0,1.016692,2.789070,0
2,2,105-45,apo25m,2M00020972+1612294,2772431905312063104,3,548.875730,0.540514,16.208181,131072,...,2,0.999707,5.054063,2,0.999854,4.235257,2,1.000000,5.116467,2
3,4,107-46_MGA,apo25m,2M00020972+1612294,2772431905312063104,2,196.888920,0.540514,16.208181,131072,...,2,0.999707,5.054063,2,0.999854,4.235257,2,1.000000,5.116467,2
4,6,100-60,apo25m,2M00021917+0142107,2738655354783502080,4,243.988720,0.579878,1.702982,0,...,1,0.644867,37.396367,0,0.617390,40.565028,0,0.654591,33.264564,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4283,5149,N7789,apo25m,2M23582919+5601109,1994738569261182080,3,77.150215,359.621650,56.019722,512,...,0,1.091919,6.546461,0,1.058801,6.993268,0,1.093822,6.524955,0
4284,5150,120+12,apo25m,2M23591060+7448266,2229206503298163584,9,1993.943200,359.794168,74.807404,131072,...,0,1.636265,1.622122,0,1.582327,1.757561,0,1.600989,1.832215,0
4285,5151,105-45,apo25m,2M23592268+1714293,2773673906774562688,3,310.058870,359.844517,17.241491,0,...,1,1.297598,3.151082,1,1.319417,2.958520,1,1.411742,2.742071,1
4286,5152,SMC12,lco25m,2M23594797-7254435,6380061394361042048,12,896.137450,359.949908,-72.912109,0,...,0,1.613165,1.768008,0,1.568500,1.902231,0,1.580845,2.026423,0


In [ ]:
# Extract ID function for the Joker data below
def extract_id(value):
    return re.search(r"b'(.+)'", str(value)).group(1)


In [3]:
#Now import Joker data and slim it down to only the necessary parameters

data = Table.read('/Users/mschochet/Desktop/Research/Testing_Docs/apJoker-metadata.fits', format='fits')
jokerdf = data.to_pandas()

jokerdf['APOGEE_ID'] = jokerdf['APOGEE_ID'].apply(extract_id)

slimmedjokerdf = jokerdf[['APOGEE_ID', 'MAP_P', 'MAP_e', 'MAP_K']].copy()
slimmedjokerdf = slimmedjokerdf.rename(columns={"MAP_P": "Period [Joker, Days]",
                                   "MAP_e": "Eccentricity [Joker]", "MAP_K": "K [Joker, km/s]"})

changedperiod = slimmedjokerdf["Period [Joker, Days]"].mul(86400)
df10 = pd.DataFrame(changedperiod)


slimmedjokerdf = slimmedjokerdf.rename(columns={"Period [Joker, Days]": "Period [Joker, Seconds]"})
slimmedjokerdf["Period [Joker, Seconds]"] = 0
slimmedjokerdf["Period [Joker, Seconds]"] = df10
slimmedjokerdf

,APOGEE_ID,"Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]"
0,2M00000002+7417074,3.145566e+06,0.419240,16.410231
1,2M00000019-1924498,3.612605e+05,0.000712,33.166533
2,2M00000032+5737103,3.145566e+06,0.419240,16.410231
3,2M00000068+5710233,1.381036e+05,0.659999,47.022936
4,2M00000133+5721163,3.145566e+06,0.419240,16.410231
...,...,...,...,...
358345,AP18341149-2302059,1.984497e+05,0.767292,0.139080
358346,FF_Aql,1.297341e+05,0.110467,23.033727
358347,FN_Aql,1.296254e+05,0.102757,85.302459
358348,GQ_Ori,1.321291e+05,0.089768,23.796041


In [31]:
#Combine Joker data with the precut CSV of Yu radii, Kiauhoku masses, and APOGEE parameters 

df3 = df_fircomb.merge(slimmedjokerdf, how='outer',left_on="SDSS ID", right_on="APOGEE_ID")
df4 = df3.query("nvisits >1")
df_wjoker = df4.drop(['Unnamed: 0', 'APOGEE_ID'], axis=1)

In [36]:
#This cell calculates a rotational period for a tidally synchronized object in seconds; in case the period from Joker doesn't exist

df_wjoker.insert(34,'Period [Rotational, seconds]',np.zeros(4288))
for i in range (len(df_wjoker["SDSS ID"])):
    vsini = df_wjoker['vsini'].iloc[i]
    Radius_km = (df_wjoker['radius'].iloc[i])* 695700
    pi = np.pi
    per = (((2 * pi) * Radius_km) / vsini)
    df_wjoker['Period [Rotational, seconds]'].iloc[i] = per

ValueError: cannot insert Period [Rotational, seconds], already exists

In [37]:
# Make columns for the combined parameter of the right hand side of the binary mass function for all our objects

df_wjoker.insert(37,'Beta [Solar Masses, Joker]',np.zeros(4288))
df_wjoker.insert(35,'Beta [Solar Masses, Rotational Period]',np.zeros(4288))

ValueError: cannot insert Beta [Solar Masses, Joker], already exists

In [34]:
#Import necessary constants
pi = np.pi
grav = 6.6743 * (10**-20)


#Calculates Beta using Joker Radius 
for i in range (len(df_wjoker["SDSS ID"])):
    K = df_wjoker['K [Joker, km/s]'].iloc[i]
    period = df_wjoker['Period [Joker, Seconds]'].iloc[i]
    e = df_wjoker['Eccentricity [Joker]'].iloc[i]
    df_wjoker['Beta [Solar Masses, Joker]'].iloc[i] = (((((K**3) * period) / (2 * pi * grav)) * 5.02785e-31) * ((1 - e)**(3/2)))



#Calculates Beta using the values for assumed rotational synchronization 
for i in range (len(df_wjoker['SDSS ID'])):
    vscatter = df_wjoker['vscatter'].iloc[i]
    period = df_wjoker['Period [Rotational, seconds]'].iloc[i]
    df_wjoker['Beta [Solar Masses, Rotational Period]'].iloc[i] = ((((vscatter**3) * period) / (2 * pi * grav)) * 5.02785e-31)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Using MIST Mass Estimates, calculate the mass of the hidden companion

In [35]:
#SM = star mass; Beta = Constants [solar mass]; CO = compact object mass

df_wjoker.insert(4,'CO Mass [Solar Masses, Joker]',np.zeros(4288))
df_wjoker.insert(5,'CO Mass [Solar Masses, Rotational Period]',np.zeros(4288))

In [9]:
#First lets get CO masses using the Joker parameters

for i in range (len(df_wjoker['SDSS ID'])):
    beta = df_wjoker['Beta [Solar Masses, Joker]'].iloc[i]
    b_3 = float(beta**3)
    b_2 = float(beta**2)
    SM_1 = df_wjoker['mist_mass'].iloc[i]
    SM_4 = float(SM_1**4)
    SM_3 = float(SM_1**3)
    SM_2 = float(SM_1**2)
    term_1_num = (((27.0 * SM_2 * beta) + (3 * (3**(1/2)) * (((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2))) + 
                      (18.0 * SM_1 * b_2) + (2.0 * b_3))**(1/3))
    term_1_dem = ((2**(1/3)) * 3)
    term_1 = term_1_num / term_1_dem
    term_2_num = (2**(1/3))*(-1)*((6.0 * SM_1 * beta) + b_2)
    term_2_dem = (((27.0 * SM_2 * beta) + (3 * (3**(1/2)) * ((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2)) + 
                      (18.0 * SM_1 * b_2) +(2.0 * b_3))**(1/3))
    term_2 = term_2_num / term_2_dem
    term_3 = (1/3) * beta
    term_4 = SM_1
    df_wjoker['CO Mass [Solar Masses, Joker]'].iloc[i] = (term_1 - term_2 + term_3)



#Now using the values for assumed rotational synchronization 

for i in range (len(df_wjoker['SDSS ID'])):
    beta = df_wjoker['Beta [Solar Masses, Rotational Period]'].iloc[i]
    b_3 = float(beta**3)
    b_2 = float(beta**2)
    SM_1 = df_wjoker['mist_mass'].iloc[i]
    SM_4 = float(SM_1**4)
    SM_3 = float(SM_1**3)
    SM_2 = float(SM_1**2)
    term_1_num = (((27.0 * SM_2 * beta) + (3 * (3**(1/2)) * (((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2))) + 
                      (18.0 * SM_1 * b_2) + (2.0 * b_3))**(1/3))
    term_1_dem = ((2**(1/3)) * 3)
    term_1 = term_1_num / term_1_dem
    term_2_num = (2**(1/3))*(-1)*((6.0 * SM_1 * beta) + b_2)
    term_2_dem = (((27.0 * SM_2 * beta) + (3 * (3**(1/2)) * ((27.0 * SM_4 * b_2) + (4.0 * b_3 * SM_3))**(1/2)) + 
                      (18.0 * SM_1 * b_2) +(2.0 * b_3))**(1/3))
    term_2 = term_2_num / term_2_dem
    term_3 = (1/3) * beta
    term_4 = SM_1
    df_wjoker['CO Mass [Solar Masses, Rotational Period]'].iloc[i] = (term_1 - term_2 + term_3)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [13]:
# Making an extra column to flag the objects that are particularly poor but still are in the sample

df_wjoker.insert(42,'Flags',np.zeros(4288))

In [16]:
# Loop through every object and check if they have the APOGEE bitmasks set (https://www.sdss4.org/dr17/irspec/apogee-bitmasks/) off for:
## TEFF_BAD, LOGG_BAD, M_H_BAD (all go into Kiauhoku, so if set, do not trust)
## RV_FAIL (failure of the RV which is a necessary parameter for our analysis, so if set, do not trust)
## PROBLEM_TARGET (ASPCAP analysis failed, so if set, do not trust)

for i in range (len(df_wjoker) - 1):
    teffbad = str(df_wjoker['ASPCAP Flags (spelled)'].iloc[i]).find("TEFF_BAD")
    loggbad = str(df_wjoker['ASPCAP Flags (spelled)'].iloc[i]).find("LOGG_BAD")
    mhbad = str(df_wjoker['ASPCAP Flags (spelled)'].iloc[i]).find("M_H_BAD")
    rvfail = str(df_wjoker['ASPCAP Flags (spelled)'].iloc[i]).find("RV_FAIL")  
    probtarg = str(df_wjoker['ASPCAP Flags (spelled)'].iloc[i]).find("PROBLEM_TARGET")   
    if((teffbad != (-1)) == True):
        df_wjoker.iat[i, 42] =  1
    elif((loggbad != (-1)) == True):
        df_wjoker.iat[i, 42] =  1
    elif((mhbad != (-1)) == True):
        df_wjoker.iat[i, 42] =  1
    elif((rvfail != (-1)) == True):
        df_wjoker.iat[i, 42] =  1
    elif((probtarg != (-1)) == True):
        df_wjoker.iat[i, 42] =  1

In [18]:
#Drop the unneccessary columns

df_finjok = df_wjoker.drop(['Gaia ID', 'Beta [Solar Masses, Joker]', 'Beta [Solar Masses, Rotational Period]'], axis=1)
df_finjok

,Field,Telescope,SDSS ID,"CO Mass [Solar Masses, Joker]","CO Mass [Solar Masses, Rotational Period]",nvisits,SNR,RA,Dec,Star Flags (bitwise),...,age_mist,flag_mist,dart_mass,age_dart,flag_dart,"Period [Rotational, seconds]","Period [Joker, Seconds]",Eccentricity [Joker],"K [Joker, km/s]",Flags
14,120+12,apo25m,2M00000662+7528598,NaN,0.013740,9.0,773.235000,0.027622,75.483292,131072.0,...,4.235257,2.0,1.000000,5.116467,2.0,247396.251012,NaN,NaN,NaN,0.0
138,100-60,apo25m,2M00005343+0040594,0.479576,0.033036,4.0,95.417015,0.222657,0.683168,8390656.0,...,2.727639,0.0,1.016692,2.789070,0.0,341669.796439,198580.084991,0.234826,54.428057,0.0
392,105-45,apo25m,2M00020972+1612294,NaN,0.074232,3.0,548.875730,0.540514,16.208181,131072.0,...,4.235257,2.0,1.000000,5.116467,2.0,180237.825442,NaN,NaN,NaN,0.0
393,107-46_MGA,apo25m,2M00020972+1612294,NaN,0.096460,2.0,196.888920,0.540514,16.208181,131072.0,...,4.235257,2.0,1.000000,5.116467,2.0,170273.082827,NaN,NaN,NaN,0.0
419,100-60,apo25m,2M00021917+0142107,0.012477,0.009545,4.0,243.988720,0.579878,1.702982,0.0,...,40.565028,0.0,0.654591,33.264564,0.0,508207.533814,132221.712112,0.239191,3.508314,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360615,N7789,apo25m,2M23582919+5601109,0.911328,0.025770,3.0,77.150215,359.621650,56.019722,512.0,...,6.993268,0.0,1.093822,6.524955,0.0,417021.795569,176730.503082,0.406749,97.596803,0.0
360723,120+12,apo25m,2M23591060+7448266,NaN,0.122002,9.0,1993.943200,359.794168,74.807404,131072.0,...,1.757561,0.0,1.600989,1.832215,0.0,161804.513112,NaN,NaN,NaN,0.0
360762,105-45,apo25m,2M23592268+1714293,NaN,0.030618,3.0,310.058870,359.844517,17.241491,0.0,...,2.958520,1.0,1.411742,2.742071,1.0,429341.440617,NaN,NaN,NaN,0.0
360830,SMC12,lco25m,2M23594797-7254435,NaN,0.093520,12.0,896.137450,359.949908,-72.912109,0.0,...,1.902231,0.0,1.580845,2.026423,0.0,110042.788609,NaN,NaN,NaN,0.0


In [23]:
#Rename the columns for future ease of use

df_final = df_finjok.rename({"Signal-to-noise" : "SNR", "J-Band Magnitude" : "J_M", "J-Band Error":"J_M_e",
                              "H-Band Magnitude" : "H_M", "H-Band Error" : "H_M_e",
                              "K-Band Magnitude" : "K_M", "K-Band Error" : "K_M_e", "Effective Temperature Error": "teff_e",
                             "Log(g) Error" : "logg_e", "M/H": "M_H", "M/H Error": "M_H_e", "Radius [Solar Radii]" : "radius",
                             "Period [Joker, Seconds]" : "P_joker", "Eccentricity [Joker]" : "e_joker", "K [Joker, km/s]" : "K_joker",
                             "Period [Rotational, seconds]" : "P_rsync", "Beta [Solar Masses, Joker]" : "RHS_joker",
                             "Beta [Solar Masses, Rotational Period]": "RHS_rsync", "CO Mass [Solar Masses, Joker]" : "CO_mass_joker", "CO Mass [Solar Masses, Rotational Period]": "CO_mass_rsync"}, axis=1)

df_final


In [ ]:
#Print out the final data frame to a csv
#df_final.to_csv('5-6+kiahoku.csv')

In [20]:
greenobjs_rp = df_final.query('CO_mass_rsync <= 1')
greenobjs_rp

,Field,Telescope,SDSS ID,CO_mass_joker,CO_mass_rsync,nvisits,SNR,RA,Dec,Star Flags (bitwise),...,age_mist,flag_mist,dart_mass,age_dart,flag_dart,P_rsync,P_joker,e_joker,K_joker,Flags
14,120+12,apo25m,2M00000662+7528598,NaN,0.013740,9.0,773.235000,0.027622,75.483292,131072.0,...,4.235257,2.0,1.000000,5.116467,2.0,247396.251012,NaN,NaN,NaN,0.0
138,100-60,apo25m,2M00005343+0040594,0.479576,0.033036,4.0,95.417015,0.222657,0.683168,8390656.0,...,2.727639,0.0,1.016692,2.789070,0.0,341669.796439,198580.084991,0.234826,54.428057,0.0
392,105-45,apo25m,2M00020972+1612294,NaN,0.074232,3.0,548.875730,0.540514,16.208181,131072.0,...,4.235257,2.0,1.000000,5.116467,2.0,180237.825442,NaN,NaN,NaN,0.0
393,107-46_MGA,apo25m,2M00020972+1612294,NaN,0.096460,2.0,196.888920,0.540514,16.208181,131072.0,...,4.235257,2.0,1.000000,5.116467,2.0,170273.082827,NaN,NaN,NaN,0.0
419,100-60,apo25m,2M00021917+0142107,0.012477,0.009545,4.0,243.988720,0.579878,1.702982,0.0,...,40.565028,0.0,0.654591,33.264564,0.0,508207.533814,132221.712112,0.239191,3.508314,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360615,N7789,apo25m,2M23582919+5601109,0.911328,0.025770,3.0,77.150215,359.621650,56.019722,512.0,...,6.993268,0.0,1.093822,6.524955,0.0,417021.795569,176730.503082,0.406749,97.596803,0.0
360723,120+12,apo25m,2M23591060+7448266,NaN,0.122002,9.0,1993.943200,359.794168,74.807404,131072.0,...,1.757561,0.0,1.600989,1.832215,0.0,161804.513112,NaN,NaN,NaN,0.0
360762,105-45,apo25m,2M23592268+1714293,NaN,0.030618,3.0,310.058870,359.844517,17.241491,0.0,...,2.958520,1.0,1.411742,2.742071,1.0,429341.440617,NaN,NaN,NaN,0.0
360830,SMC12,lco25m,2M23594797-7254435,NaN,0.093520,12.0,896.137450,359.949908,-72.912109,0.0,...,1.902231,0.0,1.580845,2.026423,0.0,110042.788609,NaN,NaN,NaN,0.0


In [24]:
greenobjs_jok = df_final.query('CO_mass_rsync > 2')
greenobjs_jok

,Field,Telescope,SDSS ID,CO_mass_joker,CO_mass_rsync,nvisits,SNR,RA,Dec,Star Flags (bitwise),...,age_mist,flag_mist,dart_mass,age_dart,flag_dart,P_rsync,P_joker,e_joker,K_joker,Flags
310300,K11_076+13,apo25m,2M19245871+4444081,2.367665,2.693888,4.0,177.96790,291.244644,44.735600,8388608.0,...,3.334529,0.0,1.329823,3.241474,0.0,2.234910e+06,153624.998474,0.206586,150.156944,0.0
326699,CygnusX_C_btx,apo25m,2M20313189+4101058,NaN,9.147087,2.0,73.54147,307.882891,41.018291,8521728.0,...,4.002896,0.0,1.200000,3.498927,1.0,8.329948e+05,NaN,NaN,NaN,0.0


In [ ]:
## #df_wjoker.CO_mass_joker
#df_wjoker.CO_mass_rsync

col_list = df_wjoker.columns
print(col_list)

df_sorted = df_wjoker[['Field', 'Telescope', 'SDSS ID', 'nvisits', 'CO_mass_joker', 'CO_mass_rsync', 'SNR',
                      'vscatter', 'radius', 'RA', 'Dec',
                      'Star Flags (bitwise)','Star Flags (spelled)', 'ASPCAP Flags (bitwise)',
                      'ASPCAP Flags (spelled)', 'teff', 'teff_e', 'logg', 'logg_e', 'vsini', 'SLs', 'M_H',
                      'M_H_e', 'Fiber Dispersion', 'P_joker', 'e_joker', 'K_joker', 'P_rsync', 'RHS_joker','RHS_rsync']]


df_sorted.to_csv('sorted_file.csv')

In [ ]:
greenobjs_joker = df_wjoker.query('`CO_mass_joker` > 2.17 & `P_joker` > 1')
greenobjs_joker

greenobjs_joker['bin'] = greenobjs_joker['K_M']
greenobjs_joker = greenobjs_joker.sort_values(by='teff', ascending=False)


#Binned Data, Processed Outside
#binned_df = pd.read_csv('APOGEE+JOKER+ESTIMATE_MASS+ERROR+BINS.csv')

binned_df=binned_df.sort_values(by='teff', ascending=False)

In [ ]:
val1=0

for row in range(len(binned_df['SDSS ID'])):
    val1 = binned_df['Bin '][row]
    if(binned_df.iloc[row, 5] == greenobjs_joker.iloc[row, 3]):
        greenobjs_joker.iloc[row, 35] = val1
    
        
greenobjs_joker['bin'] = greenobjs_joker['bin'].astype(int)
greenobjs_joker

In [ ]:
greenobjs_joker
#greenobjs_joker.to_csv('binned_joker_stars.csv')

In [ ]:
vscatter = 58.97
period = 3.6 * 86400 / (10**(-5))
rad = 2.77 * 6.95 * (10**10)

vel = (2 * np.pi * rad) / period

vsini = 30.415
inc = vsini / vel

trueinc = np.arcsin(inc)

print(inc**3)

In [ ]:
((((vscatter**3) * period) / (2 * pi * grav)) * 5.02785e-31)